In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, confusion_matrix, classification_report, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Experiment 1 - Simple Logistic Regression

In [2]:
def esa_score(phi, alpha):
    return np.dot(alpha, phi)

def threshold_crossing_rate(esa_baseline, esa_moral, tau):
    crossed = (esa_baseline < tau) & (esa_moral >= tau)
    return np.mean(crossed)

def moral_win_rate(esa_baseline, esa_moral):
    return np.mean(esa_moral > esa_baseline)

def esa_difference(esa_baseline, esa_moral):
    return np.mean(esa_moral - esa_baseline)

In [3]:
def evaluate_classification(y_true, y_pred):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred)
    }
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    return metrics

def evaluate_esa_metrics(esa_baseline, esa_moral, tau):
    return {
        'esa_diff': esa_difference(esa_baseline, esa_moral),
        'tcr': threshold_crossing_rate(esa_baseline, esa_moral, tau),
        'moral_win_rate': moral_win_rate(esa_baseline, esa_moral)
    }

In [5]:
if __name__ == '__main__':
    #read orig dataset
    df = pd.read_csv('grad admission.csv')
    
    y = df['accept_status']
    X = df.drop(columns=['accept_status'])

    n_splits = 20
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    preds = np.zeros(len(X))

    #start timer
    start_time = time.perf_counter()
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = LogisticRegression(max_iter=1000)
        model.fit(X_train, y_train)

        y_val_pred = model.predict(X_val)
        preds[val_idx] = y_val_pred

        print(f"Fold {fold+1} Baseline Accuracy:", accuracy_score(y_val, y_val_pred))

    #end time
    end_time = time.perf_counter()
    
    elapsed_time = end_time - start_time
    
    print("\nLogistic Regression Baseline")
    print("Baseline Accuracy:", accuracy_score(y, preds))
    print(f"Time taken: {elapsed_time:.6f} seconds")
    
    metrics = evaluate_classification(y, preds)
    print(metrics)

Fold 1 Baseline Accuracy: 0.9230769230769231
Fold 2 Baseline Accuracy: 0.8717948717948718
Fold 3 Baseline Accuracy: 0.8461538461538461
Fold 4 Baseline Accuracy: 0.9230769230769231
Fold 5 Baseline Accuracy: 0.7948717948717948
Fold 6 Baseline Accuracy: 0.8974358974358975
Fold 7 Baseline Accuracy: 0.9230769230769231
Fold 8 Baseline Accuracy: 0.8461538461538461
Fold 9 Baseline Accuracy: 0.9487179487179487
Fold 10 Baseline Accuracy: 0.9473684210526315
Fold 11 Baseline Accuracy: 0.9473684210526315
Fold 12 Baseline Accuracy: 0.868421052631579
Fold 13 Baseline Accuracy: 0.8947368421052632
Fold 14 Baseline Accuracy: 0.9210526315789473
Fold 15 Baseline Accuracy: 0.868421052631579
Fold 16 Baseline Accuracy: 0.8157894736842105
Fold 17 Baseline Accuracy: 0.9736842105263158
Fold 18 Baseline Accuracy: 0.9473684210526315
Fold 19 Baseline Accuracy: 0.9210526315789473
Fold 20 Baseline Accuracy: 0.868421052631579

Logistic Regression Baseline
Baseline Accuracy: 0.8972691807542262
Time taken: 0.437410 sec

# Experiment 2 - Logistic Regression on ESA-augmented Dataset 

In [14]:
if __name__ == '__main__':
    #read orig dataset
    df2 = pd.read_csv('credit_risk_dataset - ethics.csv')
    
    # Initialize LabelEncoder
    label_encoder = LabelEncoder()
    
    # Apply label encoding
    df2['person_home_ownership'] = label_encoder.fit_transform(df2['person_home_ownership'])
    df2['loan_intent'] = label_encoder.fit_transform(df2['loan_intent'])
    df2['loan_grade'] = label_encoder.fit_transform(df2['loan_grade'])
    df2['cb_person_default_on_file'] = label_encoder.fit_transform(df2['cb_person_default_on_file'])
    
    y = df2['loan_status']
    X = df2.drop(columns=['loan_status'])

    n_splits = 20
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    preds = np.zeros(len(X))

    #start timer
    start_time = time.perf_counter()
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = LogisticRegression(max_iter=1000)
        model.fit(X_train, y_train)

        y_val_pred = model.predict(X_val)
        preds[val_idx] = y_val_pred

        print(f"Fold {fold+1} ESA-Augmented Accuracy:", accuracy_score(y_val, y_val_pred))

    #end time
    end_time = time.perf_counter()
    
    elapsed_time = end_time - start_time
    
    print("\nLogistic Regression ESA-Augmented")
    print("Accuracy:", accuracy_score(y, preds))
    print(f"Time taken: {elapsed_time:.6f} seconds")
    
    metrics = evaluate_classification(y, preds)
    print(metrics)

Fold 1 ESA-Augmented Accuracy: 0.811042944785276
Fold 2 ESA-Augmented Accuracy: 0.7937384898710865
Fold 3 ESA-Augmented Accuracy: 0.805402087170043
Fold 4 ESA-Augmented Accuracy: 0.8047882136279927
Fold 5 ESA-Augmented Accuracy: 0.8004910988336402
Fold 6 ESA-Augmented Accuracy: 0.8029465930018416
Fold 7 ESA-Augmented Accuracy: 0.8103130755064457
Fold 8 ESA-Augmented Accuracy: 0.807243707796194
Fold 9 ESA-Augmented Accuracy: 0.809085328422345
Fold 10 ESA-Augmented Accuracy: 0.809085328422345
Fold 11 ESA-Augmented Accuracy: 0.7961939840392879
Fold 12 ESA-Augmented Accuracy: 0.8035604665438919
Fold 13 ESA-Augmented Accuracy: 0.8041743400859422
Fold 14 ESA-Augmented Accuracy: 0.809085328422345
Fold 15 ESA-Augmented Accuracy: 0.8041743400859422
Fold 16 ESA-Augmented Accuracy: 0.8011049723756906
Fold 17 ESA-Augmented Accuracy: 0.7992633517495396
Fold 18 ESA-Augmented Accuracy: 0.8084714548802947
Fold 19 ESA-Augmented Accuracy: 0.7986494782074892
Fold 20 ESA-Augmented Accuracy: 0.806629834254

# Experiment 3 - Logistic Regression on ESA-augmented Dataset with CatBoost

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, confusion_matrix, classification_report, log_loss
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold

In [6]:
def evaluate_classification(y_true, y_pred):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred)
    }
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    return metrics

def evaluate_regression(y_true, y_pred):
    return {
        'mse': mean_squared_error(y_true, y_pred),
        'rmse': mean_squared_error(y_true, y_pred, squared=False)
    }

def evaluate_esa_metrics(esa_baseline, esa_moral, tau):
    return {
        'esa_diff': esa_difference(esa_baseline, esa_moral),
        'tcr': threshold_crossing_rate(esa_baseline, esa_moral, tau),
        'moral_win_rate': moral_win_rate(esa_baseline, esa_moral)
    }

In [7]:
def evaluate_esa_decision_function(model, X, phi_matrix, alpha, tau_values):
    y_proba = model.predict_proba(X)[:, 1]
    y_pred = model.predict(X)
    esa_scores = np.array([esa_score(phi, alpha) for phi in phi_matrix])
    loan_status_moral = (esa_scores >= tau_values).astype(int)

    eval_moral = evaluate_classification(y_pred, loan_status_moral)
    eval_moral['roc_auc'] = roc_auc_score(y_pred, loan_status_moral)
    return eval_moral, esa_scores, loan_status_moral

In [8]:
if __name__ == '__main__':
    # Example dummy ESA weights
    alpha = np.array([0.4, 0.2, 0.3, 0.0, 0.0, 0.1])  # For consequentialism

    # Load data (replace this with your actual DataFrame)
    df = pd.read_csv('grad admission - ethics.csv')  # Assumes 6 ESA features + CST + loan_status + other inputs
    
    # Extract ESA features and CST
    esa_features = df[['severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int']].values
    tau_values = df['CST'].values
    y = df['accept_status_moral']
    X = df.drop(columns=['accept_status', 'ESA', 'CST', 'severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int', 'accept_status_moral'])

    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    catboost_params = {
        'depth': 7,
        'learning_rate': 0.19893301995319765,
        'bagging_temperature': 0.7979373495258176,
        'l2_leaf_reg': 5,
        'loss_function': 'Logloss',
        'iterations': 400,
        'grow_policy': 'Lossguide',
        'eval_metric': 'AUC',
        'verbose': False
    }

    preds = np.zeros(len(X))
    moral_preds = np.zeros(len(X))
    all_esa = np.zeros(len(X))

    #start timer
    start_time = time.perf_counter()
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        phi_val = esa_features[val_idx]
        tau_val = tau_values[val_idx]

        model = CatBoostClassifier(**catboost_params)
        model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=10)

        # Apply ESA-informed decision override
        eval_moral, esa_vals, moral_decisions = evaluate_esa_decision_function(model, X_val, phi_val, alpha, tau_val)
        preds[val_idx] = model.predict(X_val)
        moral_preds[val_idx] = moral_decisions

        print(f"Fold {fold+1}:", eval_moral)
        
    #end time
    end_time = time.perf_counter()
    
    elapsed_time = end_time - start_time

    print("\n--- Final Evaluation ---")
    print("Standard Accuracy:", accuracy_score(y, preds))
    print("Moral Accuracy:", accuracy_score(y, moral_preds))
    print("Moral Agreement:", accuracy_score(preds, moral_preds))

    
metrics = evaluate_classification(y, preds)
print(metrics)


Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        65
           1       0.58      1.00      0.73        89

    accuracy                           0.58       154
   macro avg       0.29      0.50      0.37       154
weighted avg       0.33      0.58      0.42       154


Confusion Matrix:
[[ 0 65]
 [ 0 89]]
Fold 1: {'accuracy': 0.577922077922078, 'precision': 0.577922077922078, 'recall': 1.0, 'f1_score': 0.7325102880658436, 'roc_auc': 0.5}

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        65
           1       0.58      1.00      0.73        89

    accuracy                           0.58       154
   macro avg       0.29      0.50      0.37       154
weighted avg       0.33      0.58      0.42       154


Confusion Matrix:
[[ 0 65]
 [ 0 89]]
Fold 2: {'accuracy': 0.577922077922078, 'precision': 0.577922077922078, 'recall': 1.0, 

C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  


Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        65
           1       0.58      1.00      0.73        89

    accuracy                           0.58       154
   macro avg       0.29      0.50      0.37       154
weighted avg       0.33      0.58      0.42       154


Confusion Matrix:
[[ 0 65]
 [ 0 89]]
Fold 4: {'accuracy': 0.577922077922078, 'precision': 0.577922077922078, 'recall': 1.0, 'f1_score': 0.7325102880658436, 'roc_auc': 0.5}

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        64
           1       0.58      1.00      0.74        89

    accuracy                           0.58       153
   macro avg       0.29      0.50      0.37       153
weighted avg       0.34      0.58      0.43       153


Confusion Matrix:
[[ 0 64]
 [ 0 89]]
Fold 5: {'accuracy': 0.5816993464052288, 'precision': 0.5816993464052288, 'recall': 1.0

C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

# Experiment 4 - Logistic Regression on regular Dataset with CatBoost

In [51]:
if __name__ == '__main__':
    # Example dummy ESA weights
    #alpha = np.array([0.4, 0.2, 0.3, 0.0, 0.0, 0.1])  # For consequentialism

    # Load data (replace this with your actual DataFrame)
    df4 = pd.read_csv('credit_risk_dataset.csv')  # Assumes 6 ESA features + CST + loan_status + other inputs

    # Initialize LabelEncoder
    label_encoder = LabelEncoder()

    # Apply label encoding
    df4['person_home_ownership'] = label_encoder.fit_transform(df4['person_home_ownership'])
    df4['loan_intent'] = label_encoder.fit_transform(df4['loan_intent'])
    df4['loan_grade'] = label_encoder.fit_transform(df4['loan_grade'])
    df4['cb_person_default_on_file'] = label_encoder.fit_transform(df4['cb_person_default_on_file'])
    
    # Extract ESA features and CST
    #esa_features = df[['severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int']].values
    #tau_values = df['CST'].values
    y = df['loan_status']
    X = df.drop(columns=['loan_status'])

    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    catboost_params = {
        'depth': 7,
        'learning_rate': 0.19893301995319765,
        'bagging_temperature': 0.7979373495258176,
        'l2_leaf_reg': 5,
        'loss_function': 'Logloss',
        'iterations': 400,
        'grow_policy': 'Lossguide',
        'eval_metric': 'AUC',
        'verbose': False
    }

    preds = np.zeros(len(X))
    moral_preds = np.zeros(len(X))
    all_esa = np.zeros(len(X))

    #start timer
    start_time = time.perf_counter()
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        phi_val = esa_features[val_idx]
        tau_val = tau_values[val_idx]

        model = CatBoostClassifier(**catboost_params)
        model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=10)

        # Apply ESA-informed decision override
        #eval_moral, esa_vals, moral_decisions = evaluate_esa_decision_function(model, X_val, phi_val, alpha, tau_val)
        preds[val_idx] = model.predict(X_val)
        #moral_preds[val_idx] = moral_decisions

        #print(f"Fold {fold+1}:", eval_moral)
        
    #end time
    end_time = time.perf_counter()
    
    elapsed_time = end_time - start_time

    print("\n--- Final Evaluation ---")
    print("Standard Accuracy:", accuracy_score(y, preds))
    print("Moral Accuracy:", accuracy_score(y, moral_preds))
    print("Moral Agreement:", accuracy_score(preds, moral_preds))

    
metrics = evaluate_classification(y, preds)
print(metrics)


--- Final Evaluation ---
Standard Accuracy: 0.936527423958749
Moral Accuracy: 0.7818360394094718
Moral Agreement: 0.8371443479328443

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.99      0.96     25473
           1       0.97      0.73      0.83      7108

    accuracy                           0.94     32581
   macro avg       0.95      0.86      0.90     32581
weighted avg       0.94      0.94      0.93     32581


Confusion Matrix:
[[25340   133]
 [ 1935  5173]]
{'accuracy': 0.936527423958749, 'precision': 0.974934036939314, 'recall': 0.727771525042206, 'f1_score': 0.8334138875463188, 'roc_auc': 0.8612751552114025}


# Experiment 5 - With Penalized Term

In [19]:
def evaluate_classification(y_true, y_pred):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred)
    }
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    return metrics

def evaluate_regression(y_true, y_pred):
    return {
        'mse': mean_squared_error(y_true, y_pred),
        'rmse': mean_squared_error(y_true, y_pred, squared=False)
    }

def evaluate_esa_metrics(esa_baseline, esa_moral, tau):
    return {
        'esa_diff': esa_difference(esa_baseline, esa_moral),
        'tcr': threshold_crossing_rate(esa_baseline, esa_moral, tau),
        'moral_win_rate': moral_win_rate(esa_baseline, esa_moral)
    }

In [20]:
def evaluate_esa_decision_function(model, X, phi_matrix, alpha, tau_values):
    y_proba = model.predict_proba(X)[:, 1]
    y_pred = model.predict(X)
    esa_scores = np.array([esa_score(phi, alpha) for phi in phi_matrix])
    loan_status_moral = (esa_scores >= tau_values).astype(int)

    eval_moral = evaluate_classification(y_pred, loan_status_moral)
    eval_moral['roc_auc'] = roc_auc_score(y_pred, loan_status_moral)
    return eval_moral, esa_scores, loan_status_moral

In [21]:
# Load data (replace this with your actual DataFrame)
df = pd.read_csv('grad admission - ethics.csv')  # Assumes 6 ESA features + CST + admission_status + other inputs

# Extract ESA features and CST
esa_features = df[['severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int']].values
tau_values = df['CST'].values
y = df['accept_status_moral']
X = df.drop(columns=['accept_status', 'ESA', 'CST', 'severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int', 'accept_status_moral'])


In [22]:
 preds_penalized = np.zeros(len(X))

#start timer
start_time = time.perf_counter()
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    phi_train = esa_features[train_idx]
    tau_train = tau_values[train_idx]

    moral_penalty = np.array([(tau - esa_score(phi, alpha))**2 for phi, tau in zip(phi_train, tau_train)])
    sample_weights = np.clip(1 + 5 * moral_penalty, 1, 10)

    model2 = LogisticRegression(max_iter=1000)
    model2.fit(X_train, y_train, sample_weight=sample_weights)

    y_val_pred = model2.predict(X_val)
    preds_penalized[val_idx] = y_val_pred

    print(f"Fold {fold+1} ESA Penalized Accuracy:", accuracy_score(y_val, y_val_pred))
    
    #end time
    end_time = time.perf_counter()
    
    elapsed_time = end_time - start_time

print("\n--- Final Evaluation (Logistic Regression ESA Penalized) ---")
print("ESA Penalized Accuracy:", accuracy_score(y, preds_penalized))
print(f"Time taken: {elapsed_time:.6f} seconds")
    

metrics = evaluate_classification(y, preds_penalized)
print(metrics)


Fold 1 ESA Penalized Accuracy: 1.0
Fold 2 ESA Penalized Accuracy: 1.0
Fold 3 ESA Penalized Accuracy: 1.0
Fold 4 ESA Penalized Accuracy: 1.0
Fold 5 ESA Penalized Accuracy: 1.0

--- Final Evaluation (Logistic Regression ESA Penalized) ---
ESA Penalized Accuracy: 1.0
Time taken: 0.161866 seconds

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       324
           1       1.00      1.00      1.00       445

    accuracy                           1.00       769
   macro avg       1.00      1.00      1.00       769
weighted avg       1.00      1.00      1.00       769


Confusion Matrix:
[[324   0]
 [  0 445]]
{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0, 'roc_auc': 1.0}


In [17]:
df.head

<bound method NDFrame.head of      gre_score  toefl_score  uni_rating  sop  lor  cgpa  research  \
0          337          118           4  4.5  4.5  9.65         1   
1          324          107           4  4.0  4.5  8.87         1   
2          316          104           3  3.0  3.5  8.00         1   
3          322          110           3  3.5  2.5  8.67         1   
4          314          103           2  2.0  3.0  8.21         0   
..         ...          ...         ...  ...  ...   ...       ...   
764        302          110           3  4.0  4.5  8.50         0   
765        297           99           4  3.0  3.5  7.81         0   
766        298          101           4  2.5  4.5  7.69         1   
767        300           95           2  3.0  1.5  8.22         1   
768        301           99           3  2.5  2.0  8.45         1   

     accept_status  severity_cons  dur_cons  util_cons   prin_up  prin_vi  \
0                1       0.956485   0.77200   0.979837  0.939044

# Override later decisions

In [18]:
alpha = np.array([0.4, 0.2, 0.3, 0.0, 0.0, 0.1])
esa_features = df[['severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int']].values
tau_values = df['CST'].values
preds_override = np.zeros(len(X))

#start timer
start_time = time.perf_counter()
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    phi_val = esa_features[val_idx]
    tau_val = tau_values[val_idx]

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # ESA override decision logic
    esa_vals = np.array([esa_score(phi, alpha) for phi in phi_val])
    moral_decisions = (esa_vals >= tau_val).astype(int)
    preds_override[val_idx] = moral_decisions

    print(f"Fold {fold+1} ESA Override Accuracy:", accuracy_score(y_val, moral_decisions))

#end time
end_time = time.perf_counter()
elapsed_time = end_time - start_time

print("\n--- Final Evaluation (Logistic Regression ESA Override) ---")
print("ESA Override Accuracy:", accuracy_score(y, preds_override))
print(f"Time taken: {elapsed_time:.6f} seconds")

metrics = evaluate_classification(y, preds_override)
print(metrics)

Fold 1 ESA Override Accuracy: 0.577922077922078
Fold 2 ESA Override Accuracy: 0.577922077922078
Fold 3 ESA Override Accuracy: 0.577922077922078
Fold 4 ESA Override Accuracy: 0.577922077922078
Fold 5 ESA Override Accuracy: 0.5816993464052288

--- Final Evaluation (Logistic Regression ESA Override) ---
ESA Override Accuracy: 0.5786736020806242
Time taken: 0.140122 seconds

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       324
           1       0.58      1.00      0.73       445

    accuracy                           0.58       769
   macro avg       0.29      0.50      0.37       769
weighted avg       0.33      0.58      0.42       769


Confusion Matrix:
[[  0 324]
 [  0 445]]
{'accuracy': 0.5786736020806242, 'precision': 0.5786736020806242, 'recall': 1.0, 'f1_score': 0.7331136738056013, 'roc_auc': 0.5}


C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  